In [ ]:
# base_path = '../../tagger_data/as_it_is/'
base_path = '../../gen_data/as_it_is/'

with open(base_path + 'train.src') as f:
    train_src = f.readlines()
    print(len(train_src))
with open(base_path + 'train.trg') as f:
    train_trg = f.readlines()
    print(len(train_trg))
with open(base_path + 'dev.src') as f:
    dev_src = f.readlines()
    print(len(dev_src))
with open(base_path + 'dev.trg') as f:
    dev_trg = f.readlines()
    print(len(dev_trg))
with open(base_path + 'test.src') as f:
    test_src = f.readlines()
    print(len(test_src))
with open(base_path + 'test.trg') as f:
    test_trg = f.readlines()
    print(len(test_trg))

In [ ]:
import pandas as pd

train_df = pd.DataFrame(
    {'src': train_src,
     'trg': train_trg
    })
train_df.head()

dev_df = pd.DataFrame(
    {'src': dev_src,
     'trg': dev_trg
    })
dev_df.head()

test_df = pd.DataFrame(
    {'src': test_src,
     'trg': test_trg
    })
test_df.head()

In [ ]:
print(train_df['src'][1])
train_df.src = train_df.src.str.replace('\n', '')
dev_df.src = dev_df.src.str.replace('\n', '')
test_df.src = test_df.src.str.replace('\n', '')
print(train_df['src'][1])

print(train_df['trg'][1])
train_df.trg = train_df.trg.str.replace('\n', '')
dev_df.trg = dev_df.trg.str.replace('\n', '')
test_df.trg = test_df.trg.str.replace('\n', '')
print(train_df['trg'][1])

test_df.head()

In [ ]:
# train_df.src = train_df.src.str.replace('\[P_[a-zA-Z0-9_]+\]', '<mask>')
# dev_df.src = dev_df.src.str.replace('\[P_[a-zA-Z0-9_]+\]', '<mask>')
# test_df.src = test_df.src.str.replace('\[P_[a-zA-Z0-9_]+\]', '<mask>')

# train_df.trg = train_df.trg.str.replace('\[P_[a-zA-Z0-9_]+\]', '<mask>')
# dev_df.trg = dev_df.trg.str.replace('\[P_[a-zA-Z0-9_]+\]', '<mask>')
# test_df.trg = test_df.trg.str.replace('\[P_[a-zA-Z0-9_]+\]', '<mask>')

train_df.src = train_df.src.str.replace('\[P_[a-zA-Z0-9_]+\]', '')
dev_df.src = dev_df.src.str.replace('\[P_[a-zA-Z0-9_]+\]', '')
test_df.src = test_df.src.str.replace('\[P_[a-zA-Z0-9_]+\]', '')

train_df.src = train_df.src.str.replace('  ', ' ')
dev_df.src = dev_df.src.str.replace('  ', ' ')
test_df.src = test_df.src.str.replace('  ', ' ')

# train_df.trg = train_df.trg.str.replace('\[P_[a-zA-Z0-9_]+\]', '')
# dev_df.trg = dev_df.trg.str.replace('\[P_[a-zA-Z0-9_]+\]', '')
# test_df.trg = test_df.trg.str.replace('\[P_[a-zA-Z0-9_]+\]', '')

# train_df['src'][1]
test_df.head()

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
model_name = 'facebook/bart-base'

In [ ]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
def gen(src):
    src_tknz = tokenizer(src, truncation=True, padding=True, max_length=128, return_tensors='pt').to(device)
    generated_ids = model.generate(src_tknz["input_ids"].cuda(), max_length=128)

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
for idx in range(50):
    print('src:', test_df['src'].values.tolist()[idx])
    print('trg:', test_df['trg'].values.tolist()[idx])
    
    print('pred:', gen(test_df['src'].values.tolist()[idx]))
    print('\n')

In [ ]:
test_ips = ['send me the text files.', 'look into this issue.', 'good bye.', 'okay.', 'do it.', 'yes, go ahead and remove it.', 'jon please use this resignation letter in lieu of the one sent on friday']
for test_ip in test_ips:
    print('src:', test_ip)
    print('pred:',  gen(test_ip))
    print('\n')

In [ ]:
import pickle

In [ ]:
pred_wofinetune = []
for idx in range(len(test_df['src'].values.tolist())):
    pred_wofinetune.append((gen(test_df['src'].values.tolist()[idx])))

with open('pred_wofinetune_lists.pkl', 'wb') as f:
    pickle.dump(pred_wofinetune, f)